## End-to-end MNIST training

In [1]:
%matplotlib inline
import utils; reload(utils); # init theano GPU
from utils import *
from __future__ import division, print_function

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 960 (0000:01:00.0)
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [2]:
# SETUP 
batch_size = 256 # MNIST data is small, so my computer can surely handle this
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
# MNIST contains grayscale images, so we need to expand a dim
# so that we got data in shape: (n_samples, n_channels, h, w)
X_test = np.expand_dims(X_test, 1) # second dimension
X_train = np.expand_dims(X_train, 1) # second dimension

In [5]:
X_train.shape, X_test.shape, X_train.dtype

((60000, 1, 28, 28), (10000, 1, 28, 28), dtype('uint8'))

In [6]:
y_train[:5] # see what categories we have

array([5, 0, 4, 1, 9], dtype=uint8)

In [38]:
??onehot

In [39]:
??to_categorical

In [7]:
# convert y to onehot encoding
y_train = onehot(y_train)
y_test = onehot(y_test)

In [8]:
y_train[:5]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [9]:
# normalize the input
print(X_train.shape)
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

(60000, 1, 28, 28)


In [10]:
def norm_input(x): return (x - mean_px)/std_px

In [10]:
# LINEAR MODEL
def get_lin_model():
    """This is ensentially a multi-output linear model"""
    model = Sequential([
        Lambda(norm_input, input_shape = (1, 28, 28)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
lm = get_lin_model()

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [11]:
# make keras' numpyArrayIterator
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=batch_size)
test_batches = gen.flow(X_test, y_test, batch_size=batch_size)

# new keras params:
# steps_per_epoch: Total number of steps (batches of samples) 
# before declaring one epoch finished and starting the next
# epoch.  similar for validation_steps
steps_per_epoch = int(np.ceil(batches.n/batch_size))
validation_steps = int(np.ceil(test_batches.n/batch_size))

(batches.n, test_batches.n, steps_per_epoch, validation_steps) # show dims

(60000, 10000, 235, 40)

In [12]:
# fitting data:
# to start, use default learning rate for 1 epoch and see what you get
# for linear model, we get 91% acc, not really great  
lm.fit_generator(batches, steps_per_epoch=steps_per_epoch,
                    epochs=1, 
                    validation_data=test_batches, 
                    validation_steps=validation_steps)

NameError: name 'lm' is not defined

In [13]:
# SINGLE DENSE LAYER
def get_fc_model():
    """Add a fully-connected hidden layer. Old school."""
    model = Sequential([
        Lambda(norm_input, input_shape=(1, 28, 28)),
        Flatten(),
        Dense(1024, activation='softmax'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
fc = get_fc_model()
# With 8 training epochs, I got 93% validation accuracy
# pretty good
fc.fit_generator(batches, epochs=8, 
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps, 
                 validation_data=test_batches)

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Epoch 1/8
235/235 [==============================] - 1s 4ms/step - loss: 2.0242 - acc: 0.8159 - val_loss: 1.7901 - val_acc: 0.9173
Epoch 2/8
235/235 [==============================] - 1s 3ms/step - loss: 1.6079 - acc: 0.9199 - val_loss: 1.4277 - val_acc: 0.9235
Epoch 3/8
235/235 [==============================] - 1s 3ms/step - loss: 1.2784 - acc: 0.9250 - val_loss: 1.1325 - val_acc: 0.9272
Epoch 4/8
235/235 [==============================] - 1s 3ms/step - loss: 1.0207 - acc: 0.9280 - val_loss: 0.9124 - val_acc: 0.9285
Epoch 5/8
235/235 [==============================] - 1s 3ms/step - loss: 0.8293 - acc: 0.9313 - val_loss: 0.7534 - val_acc: 0.9298
Epoch 6/8
235/235 [==============================] - 1s 3ms/step - loss: 0.6881 - acc: 0.9347 - val_loss: 0.6348 - val_acc: 0.9314
Epoch 7/8
235/235 [==============================] - 1s 3ms/step - loss: 0.5848 - acc: 0.9363 - val_loss: 0.5511 - val_acc: 0.9314
Epoch 8/8
235/235 [==============================] - 1s 3ms/step - loss: 0.5083 - a

In [15]:
# BASIC VGG-STYLE CNN
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1, 28, 28)),
        Convolution2D(32, 3, 3, activation='relu'),
        Convolution2D(32, 3, 3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64, 3, 3, activation='relu'),
        Convolution2D(64, 3, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [16]:
model = get_model()

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  """
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if __name__ == '__main__':
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automati

In [17]:
model.fit_generator(batches, epochs=1,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/1
235/235 [==============================] - 9s 39ms/step - loss: 0.1768 - acc: 0.9472 - val_loss: 0.0432 - val_acc: 0.9854


In [18]:
# change learning rate
model.optimizer.lr=0.1

In [19]:
model.fit_generator(batches, epochs=1,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/1
235/235 [==============================] - 9s 39ms/step - loss: 0.0434 - acc: 0.9870 - val_loss: 0.0298 - val_acc: 0.9898


In [20]:
# one more epoch - we overfitted this time
model.optimizer.lr = 0.05
model.fit_generator(batches, epochs=1,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/1
235/235 [==============================] - 9s 39ms/step - loss: 0.0289 - acc: 0.9911 - val_loss: 0.0245 - val_acc: 0.9913


In [21]:
# DATA AUGMENTATION
model = get_model()

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  """
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if __name__ == '__main__':
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automati

In [22]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08,
                              height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

steps_per_epoch = int(np.ceil(batches.n/batch_size))
validation_steps = int(np.ceil(test_batches.n/batch_size))

# TODO: visualize data augmentation

In [24]:
model.fit_generator(batches, epochs=4,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/4
235/235 [==============================] - 4s 15ms/step - loss: 0.1171 - acc: 0.9641 - val_loss: 0.1025 - val_acc: 0.9688
Epoch 2/4
235/235 [==============================] - 3s 14ms/step - loss: 0.0947 - acc: 0.9699 - val_loss: 0.0694 - val_acc: 0.9789
Epoch 3/4
235/235 [==============================] - 3s 14ms/step - loss: 0.0684 - acc: 0.9805 - val_loss: 0.0530 - val_acc: 0.9824
Epoch 4/4
235/235 [==============================] - 4s 15ms/step - loss: 0.0660 - acc: 0.9803 - val_loss: 0.0771 - val_acc: 0.9785


In [25]:
# BATCH NORM
def get_model_bn():
    model = Sequential([
        Lambda(norm_input, input_shape=(1, 28, 28)),
        Convolution2D(32, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32, 3, 3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64, 3, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# TODO: WHY DO WE NEED AXIS=1 for batchnorm in Conv layers?
# solution is on the forum

In [31]:
model_bn = get_model_bn()

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  """
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  import sys
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argume

In [33]:
# start with default learning rate first
model_bn.fit_generator(batches, epochs=1,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/1
235/235 [==============================] - 6s 24ms/step - loss: 0.1057 - acc: 0.9668 - val_loss: 0.2374 - val_acc: 0.9203


In [27]:
model_bn.optimizer.lr = 0.1

In [34]:
model_bn.fit_generator(batches, epochs=4,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/4
235/235 [==============================] - 6s 24ms/step - loss: 0.0809 - acc: 0.9757 - val_loss: 0.0892 - val_acc: 0.9746
Epoch 2/4
235/235 [==============================] - 5s 23ms/step - loss: 0.0787 - acc: 0.9753 - val_loss: 0.0644 - val_acc: 0.9785
Epoch 3/4
235/235 [==============================] - 5s 23ms/step - loss: 0.0688 - acc: 0.9772 - val_loss: 0.0432 - val_acc: 0.9848
Epoch 4/4
235/235 [==============================] - 6s 24ms/step - loss: 0.0629 - acc: 0.9804 - val_loss: 0.0736 - val_acc: 0.9801


In [35]:
model_bn.optimizer.lr = 0.01

In [36]:
model_bn.fit_generator(batches, epochs=4,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/4
235/235 [==============================] - 6s 24ms/step - loss: 0.0580 - acc: 0.9820 - val_loss: 0.0596 - val_acc: 0.9812
Epoch 2/4
235/235 [==============================] - 5s 23ms/step - loss: 0.0604 - acc: 0.9815 - val_loss: 0.0466 - val_acc: 0.9875
Epoch 3/4
235/235 [==============================] - 5s 23ms/step - loss: 0.0533 - acc: 0.9829 - val_loss: 0.0663 - val_acc: 0.9840
Epoch 4/4
235/235 [==============================] - 6s 24ms/step - loss: 0.0559 - acc: 0.9819 - val_loss: 0.0546 - val_acc: 0.9805


In [ ]:
model_bn.optimizer.lr = 0.001

In [ ]:
model_bn.fit_generator(batches, epochs=8,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

In [40]:
# BATCHNORM + DROPOUT + DATA AUGMENTATION
# Combine all the good stuff so far
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1, 28, 28)),
        Convolution2D(32, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32, 3, 3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64, 3, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [41]:
model_bn_do = get_model_bn_do()

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  del sys.path[0]
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specifie

In [111]:
# should ALWAYS use the default learning rates for a couple of epoch
model_bn_do.fit_generator(batches, epochs=1,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/1
235/235 [==============================] - 6s 24ms/step - loss: 0.0475 - acc: 0.9858 - val_loss: 0.0296 - val_acc: 0.9906


In [43]:
model_bn_do.optimizer.lr = 0.1 # shift-M to merge in command mode
model_bn_do.fit_generator(batches, epochs=4,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/4
235/235 [==============================] - 6s 24ms/step - loss: 0.1633 - acc: 0.9491 - val_loss: 0.2016 - val_acc: 0.9391
Epoch 2/4
235/235 [==============================] - 6s 23ms/step - loss: 0.1192 - acc: 0.9621 - val_loss: 0.0690 - val_acc: 0.9777
Epoch 3/4
235/235 [==============================] - 6s 23ms/step - loss: 0.0986 - acc: 0.9695 - val_loss: 0.0738 - val_acc: 0.9801
Epoch 4/4
235/235 [==============================] - 6s 24ms/step - loss: 0.0924 - acc: 0.9725 - val_loss: 0.0704 - val_acc: 0.9781


In [44]:
model_bn_do.optimizer.lr = 0.01 # shift-M to merge in command mode
model_bn_do.fit_generator(batches, epochs=8,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/8
235/235 [==============================] - 6s 24ms/step - loss: 0.0887 - acc: 0.9726 - val_loss: 0.0529 - val_acc: 0.9855
Epoch 2/8
235/235 [==============================] - 5s 23ms/step - loss: 0.0811 - acc: 0.9737 - val_loss: 0.0658 - val_acc: 0.9797
Epoch 3/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0783 - acc: 0.9753 - val_loss: 0.0821 - val_acc: 0.9754
Epoch 4/8
235/235 [==============================] - 6s 24ms/step - loss: 0.0726 - acc: 0.9782 - val_loss: 0.0339 - val_acc: 0.9898
Epoch 5/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0656 - acc: 0.9805 - val_loss: 0.0501 - val_acc: 0.9859
Epoch 6/8
235/235 [==============================] - 5s 23ms/step - loss: 0.0625 - acc: 0.9800 - val_loss: 0.0410 - val_acc: 0.9883
Epoch 7/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0571 - acc: 0.9831 - val_loss: 0.0416 - val_acc: 0.9852
Epoch 8/8
235/235 [==============================] - 6s 24ms/step - loss: 0.

In [46]:
model_bn_do.optimizer.lr = 0.001 # shift-M to merge in command mode
model_bn_do.fit_generator(batches, epochs=12,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)

Epoch 1/12
235/235 [==============================] - 6s 24ms/step - loss: 0.0491 - acc: 0.9853 - val_loss: 0.0270 - val_acc: 0.9918
Epoch 2/12
235/235 [==============================] - 6s 23ms/step - loss: 0.0574 - acc: 0.9827 - val_loss: 0.0300 - val_acc: 0.9910
Epoch 3/12
235/235 [==============================] - 6s 23ms/step - loss: 0.0518 - acc: 0.9846 - val_loss: 0.0307 - val_acc: 0.9895
Epoch 4/12
235/235 [==============================] - 6s 24ms/step - loss: 0.0451 - acc: 0.9863 - val_loss: 0.0402 - val_acc: 0.9875
Epoch 5/12
235/235 [==============================] - 6s 23ms/step - loss: 0.0444 - acc: 0.9855 - val_loss: 0.0343 - val_acc: 0.9898
Epoch 6/12
235/235 [==============================] - 5s 23ms/step - loss: 0.0471 - acc: 0.9856 - val_loss: 0.0337 - val_acc: 0.9906
Epoch 7/12
235/235 [==============================] - 6s 23ms/step - loss: 0.0444 - acc: 0.9862 - val_loss: 0.0335 - val_acc: 0.9887
Epoch 8/12
235/235 [==============================] - 6s 24ms/step - 

In [48]:
# ENSEMBLE ACCROSS ALL THESE MODELS
# TODO: MAKE 6 MODELS LIKE THIS and average them
def fit_model():
    model_bn_do = get_model_bn_do()
    model_bn_do.fit_generator(batches, epochs=8,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)
    
    model_bn_do.optimizer.lr = 0.1 
    model_bn_do.fit_generator(batches, epochs=4,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)
    
    model_bn_do.optimizer.lr = 0.01 
    model_bn_do.fit_generator(batches, epochs=8,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)
    
    model_bn_do.optimizer.lr = 0.001 
    model_bn_do.fit_generator(batches, epochs=8,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps, 
                validation_data=test_batches)
    return model_bn_do

In [49]:
models = [fit_model() for i in range(6)]

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  del sys.path[0]
/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specifie

Epoch 1/8
235/235 [==============================] - 6s 24ms/step - loss: 0.4026 - acc: 0.8799 - val_loss: 7.0570 - val_acc: 0.1246
Epoch 2/8
235/235 [==============================] - 6s 23ms/step - loss: 0.1552 - acc: 0.9532 - val_loss: 0.6693 - val_acc: 0.7895
Epoch 3/8
235/235 [==============================] - 6s 23ms/step - loss: 0.1198 - acc: 0.9640 - val_loss: 0.0700 - val_acc: 0.9805
Epoch 4/8
235/235 [==============================] - 6s 24ms/step - loss: 0.1145 - acc: 0.9666 - val_loss: 0.0591 - val_acc: 0.9852
Epoch 5/8
235/235 [==============================] - 5s 23ms/step - loss: 0.0952 - acc: 0.9719 - val_loss: 0.0682 - val_acc: 0.9773
Epoch 6/8
235/235 [==============================] - 6s 24ms/step - loss: 0.0842 - acc: 0.9741 - val_loss: 0.0629 - val_acc: 0.9793
Epoch 7/8
235/235 [==============================] - 6s 24ms/step - loss: 0.0847 - acc: 0.9738 - val_loss: 0.0344 - val_acc: 0.9883
Epoch 8/8
235/235 [==============================] - 6s 24ms/step - loss: 0.

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_9 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Epoch 1/8
235/235 [==============================] - 6s 24ms/step - loss: 0.4016 - acc: 0.8803 - val_loss: 7.8458 - val_acc: 0.1223
Epoch 2/8
235/235 [==============================] - 6s 24ms/step - loss: 0.1648 - acc: 0.9490 - val_loss: 0.3942 - val_acc: 0.8840
Epoch 3/8
235/235 [==============================] - 6s 23ms/step - loss: 0.1255 - acc: 0.9616 - val_loss: 0.0975 - val_acc: 0.9676
Epoch 4/8
235/235 [==============================] - 6s 24ms/step - loss: 0.1056 - acc: 0.9661 - val_loss: 0.0566 - val_acc: 0.9816
Epoch 5/8
235/235 [==============================] - 6s 24ms/step - loss: 0.0966 - acc: 0.9712 - val_loss: 0.0584 - val_acc: 0.9812
Epoch 6/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0867 - acc: 0.9739 - val_loss: 0.0729 - val_acc: 0.9766
Epoch 7/8
235/235 [==============================] - 5s 23ms/step - loss: 0.0737 - acc: 0.9769 - val_loss: 0.0512 - val_acc: 0.9816
Epoch 8/8
235/235 [==============================] - 6s 24ms/step - loss: 0.

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_10 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


...

Epoch 1/8
235/235 [==============================] - 6s 24ms/step - loss: 0.4191 - acc: 0.8768 - val_loss: 8.2472 - val_acc: 0.1223
Epoch 2/8
235/235 [==============================] - 5s 23ms/step - loss: 0.1573 - acc: 0.9507 - val_loss: 0.2563 - val_acc: 0.9191
Epoch 3/8
235/235 [==============================] - 6s 23ms/step - loss: 0.1167 - acc: 0.9644 - val_loss: 0.0792 - val_acc: 0.9754
Epoch 4/8
235/235 [==============================] - 6s 24ms/step - loss: 0.1060 - acc: 0.9688 - val_loss: 0.0643 - val_acc: 0.9797
Epoch 5/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0818 - acc: 0.9751 - val_loss: 0.0500 - val_acc: 0.9836
Epoch 6/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0808 - acc: 0.9742 - val_loss: 0.0548 - val_acc: 0.9809
Epoch 7/8
235/235 [==============================] - 6s 24ms/step - loss: 0.0778 - acc: 0.9764 - val_loss: 0.0389 - val_acc: 0.9883
Epoch 8/8
235/235 [==============================] - 6s 24ms/step - loss: 0.

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_12 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Epoch 1/8
235/235 [==============================] - 6s 24ms/step - loss: 0.4188 - acc: 0.8760 - val_loss: 8.3897 - val_acc: 0.1223
Epoch 2/8
235/235 [==============================] - 5s 23ms/step - loss: 0.1586 - acc: 0.9516 - val_loss: 0.2203 - val_acc: 0.9359
Epoch 3/8
235/235 [==============================] - 6s 23ms/step - loss: 0.1222 - acc: 0.9622 - val_loss: 0.0891 - val_acc: 0.9676
Epoch 4/8
235/235 [==============================] - 6s 24ms/step - loss: 0.1041 - acc: 0.9696 - val_loss: 0.0674 - val_acc: 0.9781
Epoch 5/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0860 - acc: 0.9740 - val_loss: 0.0629 - val_acc: 0.9805
Epoch 6/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0869 - acc: 0.9737 - val_loss: 0.0606 - val_acc: 0.9773
Epoch 7/8
235/235 [==============================] - 5s 23ms/step - loss: 0.0745 - acc: 0.9777 - val_loss: 0.0532 - val_acc: 0.9816
Epoch 8/8
235/235 [==============================] - 6s 24ms/step - loss: 0.

/home/nguyentu/anaconda3/envs/py27_tf_cv/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_13 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Epoch 1/8
235/235 [==============================] - 6s 24ms/step - loss: 0.4120 - acc: 0.8769 - val_loss: 6.9922 - val_acc: 0.1230
Epoch 2/8
235/235 [==============================] - 6s 23ms/step - loss: 0.1531 - acc: 0.9549 - val_loss: 0.1930 - val_acc: 0.9352
Epoch 3/8
235/235 [==============================] - 6s 23ms/step - loss: 0.1228 - acc: 0.9646 - val_loss: 0.0909 - val_acc: 0.9715
Epoch 4/8
235/235 [==============================] - 6s 24ms/step - loss: 0.0973 - acc: 0.9701 - val_loss: 0.0674 - val_acc: 0.9773
Epoch 5/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0885 - acc: 0.9727 - val_loss: 0.0547 - val_acc: 0.9840
Epoch 6/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0796 - acc: 0.9759 - val_loss: 0.0422 - val_acc: 0.9848
Epoch 7/8
235/235 [==============================] - 6s 23ms/step - loss: 0.0884 - acc: 0.9743 - val_loss: 0.0520 - val_acc: 0.9855
Epoch 8/8
235/235 [==============================] - 6s 24ms/step - loss: 0.

In [51]:
HOME_DIR = os.getenv('HOME')
path = HOME_DIR + '/data/mnist/'
model_path = path + 'models/'

In [53]:
for i, m in enumerate(models):
    m.save_weights(model_path+'cnn-mnist-'+str(i)+'.pkl')


In [54]:
evals = np.array([m.evaluate(X_test, y_test, batch_size=256) for m in models])

10000/10000 [==============================] - 0s 36us/step


In [55]:
evals.mean(axis=0)

array([0.0193, 0.9937])

In [56]:
all_preds = np.stack([m.predict(X_test, batch_size=256) for m in models])

In [57]:
all_preds.shape

(6, 10000, 10)

In [92]:
avg_preds = all_preds.mean(axis=0).astype(np.float64)
avg_preds.shape, y_test.shape

((10000, 10), (10000, 10))

In [108]:
# keras metric function takes (y_true, y_pred)
# it returns array of size [n_samples] with interger values in [0, 1]
# need to do eval() and mean() to get the accuracy over the whole set
ensemble_acc = keras.metrics.categorical_accuracy(
    y_test, avg_preds).eval().mean()

In [109]:

print(ensemble_acc.shape, avg_preds.shape, y_test.shape)
print(y_test[0,:], avg_preds[0,:])
print(y_test.dtype, avg_preds.dtype)
print(ensemble_acc)

() (10000, 10) (10000, 10)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [4.7674e-07 4.2028e-07 1.1153e-07 2.0570e-07 4.2358e-08 9.1133e-09 3.0495e-10 9.9999e-01 1.4833e-09
 5.5657e-06]
float64 float64
0.9963


In [110]:
# deeper look into the categorical_accuracy function in keras
from keras.metrics import *
res = categorical_accuracy(
    np.array([[0, 1.0, 0], [1.0, 0.0, 0]], dtype=float), 
    np.array([[0, 1.0, 0], [0.0, 1.0, 0]], dtype=float)).eval()
res.mean()

0.5

In [ ]:
# TODO: TRAIN FOR LONGER TIME AND PARAM SEARCH FOR THE DATA
